## PageRank

A practical undestanding of PageRank and Personalised PageRank aglorithms.

Copyright 2010-2021 Commonwealth Scientific and Industrial Research Organisation (CSIRO).

All Rights Reserved.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

### Generate Random Graph

Graph has 5 nodes and two nodes are connected with an edge with probability given by **p**.

In [ ]:
G = nx.generators.random_graphs.fast_gnp_random_graph(n=5, p=0.5, seed=42)

Visualise the graph

In [ ]:
nx.draw(G, node_size=1000, with_labels=True, font_color='w', font_size=16)

Get the graph adjacency matrix; the graph is undirected so A is symmetric.

In [ ]:
A = nx.adjacency_matrix(G).todense()

In [ ]:
A

## What do powers of the adjacency matrix, $A^n$, represent?

What does $A^2$ look like?

In [ ]:
A @ A

The matrix $A^2$ tells us for each node in the graph in how many different ways we can reach other nodes in the graph in $2$ steps.

In [ ]:
A @ A @ A

The matrix $A^3$ tells us for each node in the graph how many different ways we can reach other nodes in the graph in 3 steps.

So, the matrix $A^n$ stores the number of paths of length $n$ for every pair of nodes in the graph.

### The transition matrix for a random walker

Consider a random walker that starts at some node in the graph and then follows edges uniformly at random.

What is the probability of the walker being at node $v$ after $n$ steps?

We can calculate the transition probabilities for any node using $AD^{-1}$ where $D$ is the degree matrix.

In [ ]:
D_inv = 1.0 / A.sum(axis=1)
D_inv = np.diagflat(D_inv)
D_inv

In [ ]:
A

Now we can calculate the transition matrix.

In [ ]:
P = A @ D_inv

In [ ]:
P

Entry $i, j$ in $P$ tells us the probability of a random walker moving from $j$ to $i$ in one step.

What happens if we take powers of $P$, e.g, $P^n$ for $n>1$?

In [ ]:
from numpy.linalg import matrix_power as mp

In [ ]:
mp(P, 2)

It tell us the probability of a random walker taking $2$ random steps of being found at node $i$ if he started at node $j$.

So, $P^n$ tells us the probability of finding the random walker at node $i$ after $n$ steps.

So, what happens if I keep increasing n?

In [ ]:
mp(P, 3)

In [ ]:
mp(P, 4)

In [ ]:
mp(P, 10)

In [ ]:
mp(P, 11)

In [ ]:
mp(P, 100)

In [ ]:
mp(P, 101)

In [ ]:
mp(P, 102)

### Converged to the stationary distribution!

It turns out that it does not matter what node the random walker started from.

In [ ]:
P_s = mp(P, 100)

Consider starting a walk at node 1. We can denote this using an indicator vector $i$.

In [ ]:
i = np.zeros((5,1))
i[1, 0] = 1
i

The probability of finding the walker at each of the $5$ nodes is given by $P_s i$

In [ ]:
P_s @ i

which happens to be the $i$-th column of $P_s$.

Note that we can calculate the entries in the above vector analytically by taking the ratio of the node's degree divided by 2 times the number of edges in the graph.

Our graph has 6 edges and the degree of node 0 is 3 which gives $3/(2*6)=0.25$.

We can calculate PageRank using Networkx building method.

In [ ]:
# alpha is a dampening factor that is usually set to a value less than 1. Here we set it to 1 to
# match the above calculation.
nx.pagerank(G, alpha=1.0)

We can think of these values as indicating the importance of a node. The more important nodes are those where the random walker has a higher probability of ending his walk.

## Personalised PageRank

The problem with PageRank is that the stationary distribution is independent of the walker's starting node. Personalised PageRank corrects for this by introducing a teleport probability that indicates the probability of the walker jumping (or teleporting) back to the starting node at each step.

Let's calculate the Personalised PageRank stationary distribution using a closed form solution.

In [ ]:
alpha = 0.1
I = np.identity(5)

In [ ]:
PP_s = alpha * mp(( I - ( 1- alpha)* P), -1)

In [ ]:
PP_s

Note that the columns are now different! **So, the stationary distribution depends on the starting node**.

### Personalised PageRank

Solve: $\pi_{ppr}(i_x)=(1-\alpha)\bar{A}\pi_{ppr}(i_x)+\alpha i_x$.

We want to solve for $\pi_{ppr}(i_x)$ so we move some of the terms around as follows,

Let $\bar{A} = AD^{-1}$ or any other probability transition matrix.

$\pi_{ppr}(i_x)=(1-\alpha)\bar{A}\pi_{ppr}(i_x)+\alpha i_x \Rightarrow$ 

$\pi_{ppr}(i_x)-(1-\alpha)\bar{A}\pi_{ppr}(i_x)=\alpha i_x \Rightarrow$

$\left(I_n-(1-\alpha)\bar{A}\right)\pi_{ppr}(i_x)=\alpha i_x \Rightarrow$

$\frac{\left(I_n-(1-\alpha)\bar{A}\right)\pi_{ppr}(i_x)}{\left(I_n-(1-\alpha)\bar{A}\right)}=\frac{\alpha i_x}{\left(I_n-(1-\alpha)\bar{A}\right)} \Rightarrow$

$\pi_{ppr}(i_x)=\alpha \left(I_n-(1-\alpha)\bar{A}\right)^{-1}i_x$